In [1]:
import numpy as np
import networkx as nx
import pickle
import pandas as pd 
import glob
import json
from tqdm import tqdm

import graph_tool.all as gt
from utilities import build_CI_rank


users_info = 'PATH TO USERS INFO'
save = False

# Top 1000 users by CI

In [ ]:
validated_networks = ['Full.gt','Left.gt','Right.gt','Full_only_lr.gt']
names = ['Full','Left','Right','Full_version2']

In [ ]:
top_users_ci = []
count_ = 0

for V in validated_networks:
    graph = gt.load_graph(f'PATH TO VALIDATED NETWORKS/{V}')
    ci_ranking = build_CI_rank(graph, graph_file=None)

    # Set or list of top 1000 user ids
    user_set = [i[0] for i in ci_ranking['out_id'][:1000]]
    
    # Access the 'id' vertex property
    id_property = graph.vertex_properties['id']    
    # Get NumPy arrays for source and target vertices
    edges = graph.get_edges()
    sources = [id_property[edge[0]] for edge in edges]
    targets = [id_property[edge[1]] for edge in edges] 

        
    # Access the edge property map containing weights
    weight_property = graph.edge_properties['weight']
    # Get NumPy array for edge weights
    weights = weight_property.get_array()
    
    # Check if source or target is in the set of users
    user_mask = np.isin(sources, user_set) | np.isin(targets, user_set)
    
    # Count total edges and sum weights
    den = len(graph.get_edges())
    den2 = np.sum(weights)
    
    # Count edges involving users and sum weights
    num = np.sum(user_mask)
    num2 = np.sum(weights[user_mask])

    
    print(f'The top 1000 influencers account for {num/den:.4f} of the unique connections in the {names[count_]}')
    print(f'The top 1000 influencers account for {num2/den2:.4f} of the connections in the {names[count_]}\n')
    
    count_ += 1
    top_users_ci.extend(user_set)
    
top_users_ci = set(top_users_ci)
##
print(f'We take to classify the top {len(top_users_ci)} users.')

In [ ]:
top_users_ci

In [ ]:
missing_ol = set(missing_ol)

In [ ]:
len(missing_ol)

In [ ]:
to_classify = []
for line in open(users_info):
    user_ = json.loads(line)
    user_id = next(iter(user_.keys()), None)
    if user_id in missing_ol:
        followers = user_[user_id]['followers_count']
        scree_name = user_[user_id]['screen_name']
        name = user_[user_id]['name']
        other_urls = user_[user_id].get('url','None')
        profile = 'https://twitter.com/'+user_[user_id]['screen_name']
        to_classify.append([user_id,followers,scree_name,name,other_urls,profile])
    if len(to_classify)==len(missing_ol):
        break
to_classify  = pd.DataFrame(to_classify,columns =['User id','n° Followers','Scree_name','Name','Url','Profile'])
if save: 
    to_classify.to_csv('PATH TO OL CLASSIFIED')

# At this point all the user are hand labelled and classified as OL or sources. 

### Original categories

In [ ]:
categories = pd.read_csv('PATH TO CATEGORIES',index_col=False)

In [ ]:
txt = 'fox'
for j,i in categories.iterrows():
    if txt.lower() in i['url']:
        print(i['url'],i['type'])
    elif txt.lower() in i['username']:
        print(i['username'],i['type'])
    elif txt.lower() in i['name']:
        print(i['name'],i['type'])

In [ ]:
categories.groupby('type').count()

### Classified users

In [ ]:
# Here is the file of labbeled users 
top_ci = pd.read_csv('PATH TO OL CLASSIFIED')
# Replace NaN with zeros
top_ci['S'] = top_ci['S'].fillna(0)


In [ ]:
OL = top_ci[(top_ci.OL==1) & (top_ci.S==0)]
influencers = top_ci[(top_ci.OL==0) & (top_ci.S==0)]
sources_ = top_ci[(top_ci.OL==0) & (top_ci.S==1)]
print('N° Influencers ', len(influencers))
print('N° OL ', len(OL))
print('N° Sources ', len(sources_))

In [ ]:
set(OL) &  set(influencers)

In [ ]:
from  utilities import build_CI_rank,get_classes
import graph_tool.all as gt
validated_networks = ['Full.gt','Left.gt','Right.gt','Full_only_lr.gt']
for G,category in zip(validated_networks,['all','left','right','all_2']):
    print(G,category)
    
    V = gt.load_graph(f'PATH TO VALIDATED GRAPH/{G}')
    
    OL,OLI,I,A,S =  get_classes(V,category=category,top=1000)
    print('\n')